<a href="https://colab.research.google.com/github/HadeerMahdy19/Gaze-Estimation-machine-learning/blob/main/notebooks/Gaze_estimation_Machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import  RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors
from sklearn import linear_model
from sklearn.multioutput import MultiOutputRegressor


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read Dataset

In [9]:
tes_data = pd.read_csv('/content/drive/MyDrive/ITI/Projects/Gaze Estimation - ml1 project/yaw_pitch_roll_dataset.csv')

In [10]:
data

,x,y,z,yaw,pitch,roll,img
0,33.0,25.0,34.0,1.044306,-22.874239,4.908886,image00002
1,55.0,41.0,38.0,68.155240,26.932743,17.243670,image00004
2,49.0,29.0,28.0,50.485413,-10.579652,-13.570645,image00006
3,38.0,29.0,25.0,17.143373,-10.048455,-21.392782,image00008
4,55.0,16.0,13.0,68.640550,-50.544582,-59.207973,image00010
...,...,...,...,...,...,...,...
1995,31.0,29.0,35.0,-4.035367,-11.293093,6.022806,image04358
1996,5.0,28.0,36.0,-81.288440,-13.327947,10.081747,image04363
1997,60.0,5.0,4.0,82.005560,-82.961685,-86.483290,image04364
1998,10.0,24.0,41.0,-68.249340,-24.090857,25.869925,image04365


## EDA
- Find duplicates
- Find Nulls


No duplicates in the dataset

In [25]:
data.duplicated().sum()

0

dataset statistic info

In [26]:
data.describe()

,x,y,z,yaw,pitch,roll
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,32.800000,30.557000,32.099000,0.806547,-6.138931,-1.975500
std,12.168814,7.114426,7.778589,37.482043,35.988570,41.024606
min,-1.000000,-1.000000,-1.000000,-351.226930,-713.745100,-972.272800
25%,27.000000,28.000000,29.000000,-16.859081,-14.811133,-10.015671
50%,33.000000,30.000000,32.000000,1.128854,-7.005438,-1.036931
75%,39.000000,33.000000,35.000000,19.177740,2.302842,7.699269
max,66.000000,66.000000,66.000000,187.786940,893.746500,333.999700


No null are found

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       2000 non-null   float64
 1   y       2000 non-null   float64
 2   z       2000 non-null   float64
 3   yaw     2000 non-null   float64
 4   pitch   2000 non-null   float64
 5   roll    2000 non-null   float64
 6   img     2000 non-null   object 
dtypes: float64(6), object(1)
memory usage: 109.5+ KB


## Split data into train, validate and test sets

In [12]:
train, validate, test = np.split(data.sample(frac=1, random_state=1), [int(.6*len(data)), int(.8*len(data))])

In [13]:
print(train.shape)
print(validate.shape)
print(test.shape)


(1200, 7)
(400, 7)
(400, 7)


In [30]:
x_train, x_validate, x_test  = train.iloc[:, :3], validate.iloc[:, :3], test.iloc[:, :3]
print(x_train.shape)
print(x_validate.shape)
print(x_test.shape)


(1200, 3)
(400, 3)
(400, 3)


In [31]:
y_train, y_validate, y_test = train.iloc[:, 3:-1], validate.iloc[:, 3:-1], test.iloc[:, 3:-1]
print(y_train.shape)
print(y_validate.shape)
print(y_test.shape)

(1200, 3)
(400, 3)
(400, 3)


## Model training
#### Training different models on our dataset

We'll train six models:
- Linear regression
- BayesianRidge regressor        ***done***   -> needs wrapper 
- DecisionTreeRegressor          ***done***
- Support vector machine (SVR)   ***done***   -> needs wrapper 
- RandomForestRegressor
- GradientBoostingRegressor
- AdaBoostRegressor
- knn                            ***done***

### 1- Use cross validation to choose best model 

In [67]:
def cross_val_repeated(model):
  # define the evaluation procedure
  cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

  # evaluate the model and collect the scores
  n_scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

  # force the scores to be positive
  n_scores = np.absolute(n_scores)
  # print(f'MAE: {np.mean(n_scores):.3f}')
  return np.mean(n_scores)


In [61]:
cv_scores_dict = {}

In [68]:
# linear regression Models

br_model = linear_model.BayesianRidge()
br_wrapper = MultiOutputRegressor(br_model)
cv_scores_dict['BayesianRidge'] = cross_val_repeated(br_wrapper)

# Tree Models
dt_model = DecisionTreeRegressor()
cv_scores_dict['DecisionTreeRegressor'] = cross_val_repeated(dt_model)

rf_model = RandomForestRegressor(n_estimators = 100, max_features = 'sqrt', max_depth = 5, random_state = 18)
cv_scores_dict['RandomForestRegressor'] = cross_val_repeated(rf_model)

# SVM Model
svm_reg_model = svm.SVR()
svm_wrapper = MultiOutputRegressor(svm_reg_model)
cv_scores_dict['SVR'] = cross_val_repeated(svm_wrapper)

# KNN Model
knn_model = neighbors.KNeighborsRegressor(5, weights='uniform')       
cv_scores_dict['KNN'] = cross_val_repeated(knn_model)


### Bayesian optimizer for best hyper-parameters

In [43]:
import BayesianOptimization

In [37]:
# Gradient Boosting Machine
def gbm_cl_bo(max_depth, max_features, learning_rate, n_estimators, subsample):
    params_gbm = {}
    params_gbm['max_depth'] = round(max_depth)
    params_gbm['max_features'] = max_features
    params_gbm['learning_rate'] = learning_rate
    params_gbm['n_estimators'] = round(n_estimators)
    params_gbm['subsample'] = subsample
    scores = cross_val_score(GradientBoostingClassifier(random_state=123, **params_gbm),
                             X_train, y_train, scoring=acc_score, cv=5).mean()
    score = scores.mean()
    return score
# Run Bayesian Optimization
start = time.time()
params_gbm ={
    'max_depth':(3, 10),
    'max_features':(0.8, 1),
    'learning_rate':(0.01, 1),
    'n_estimators':(80, 150),
    'subsample': (0.8, 1)
}
gbm_bo = BayesianOptimization(gbm_cl_bo, params_gbm, random_state=111)
gbm_bo.maximize(init_points=20, n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))

## Hyper-parameter Tuning

## Model Testing